## 创建Tensor

导入Pytorch

In [1]:
import torch

创建一个2×3的未初始化的Tensor

In [2]:
x = torch.empty(2,3)
print(x)

tensor([[2.7593e-06, 4.1342e-05, 1.0430e-08],
        [1.3602e-05, 5.2117e-11, 4.2330e+21]])


创建一个2×3的随机初始化的Tensor

In [4]:
x = torch.rand(2,3)
print(x)

tensor([[0.7726, 0.7669, 0.4244],
        [0.5425, 0.5304, 0.6620]])


创建一个2×3的long型全0的Tensor

In [7]:
x = torch.zeros(2,3,dtype=torch.long)
print(x)

tensor([[0, 0, 0],
        [0, 0, 0]])


直接根据数据创建

In [8]:
x = torch.tensor([[5.5,3],[2.2,5]])
print(x)

tensor([[5.5000, 3.0000],
        [2.2000, 5.0000]])


通过现有的Tensor来创建  
此方法会默认重用输入Tensor的一些属性，例如数据类型，除非自定义数据

In [11]:
# 返回的tensor默认具有相同的torch.dtype和torch.device
x = x.new_ones(2,3,dtype=torch.float64)
print(x)

# 指定新的数据类型
x = torch.randn_like(x,dtype=torch.float)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[-0.1414, -0.7024, -0.4585],
        [ 0.9377, -1.1660,  1.0060]])


通过shape或者size()来获取Tensor的形状

In [12]:
# 返回的torch.Size其实就是一个tuple，支持所有的tuple的操作
print(x.size())
print(x.shape)

torch.Size([2, 3])
torch.Size([2, 3])


## Tensor的相关操作

* 算术操作

在PyTorch中，同一种操作可能有很多种形式，下面用加法作为例子

In [13]:
# 加法形式一
x = torch.rand(2,3)
y = torch.rand(2,3)
print(x + y)

# 加法形式二
print(torch.add(x,y))

# 加法形式三, inplace(原地操作)
y.add_(x)
print(y)

# PyTorch操作的inplace版本都有后缀“_”，例如x.copy_(y)，x.t_()

tensor([[1.2214, 0.3709, 1.4449],
        [1.0109, 0.9053, 0.8326]])
tensor([[1.2214, 0.3709, 1.4449],
        [1.0109, 0.9053, 0.8326]])
tensor([[1.2214, 0.3709, 1.4449],
        [1.0109, 0.9053, 0.8326]])


* 索引

我们还可以使用类似NumPy的索引操作来访问Tensor的一部分  
需要注意的是：**索引出来的结果与原数据共享内存，也即修改一个，另一个会跟着修改**

In [14]:
print(x[0,:])
y = x[0,:]
y += 1
print(y)
print(x[0,:])

tensor([0.3501, 0.1389, 0.6455])
tensor([1.3501, 1.1389, 1.6455])
tensor([1.3501, 1.1389, 1.6455])


* 改变形状

用view()来改变Tensor的形状

In [16]:
y = x.view(6)
z = x.view(-1,2)
print(x.size(),y.size(),z.size())

torch.Size([2, 3]) torch.Size([6]) torch.Size([3, 2])


注意view()返回的新Tensor与源Tensor虽然可能有不同的size，但是是共享data的，也即更改其中的一个，另外一个也会跟着改变。(顾名思义，view仅仅是改变了对这个张量的观察角度，内部数据并未改变)

In [19]:
print(y)
x += 1
print(x)
print(y)# y也加了1

tensor([2.3501, 2.1389, 2.6455, 1.9278, 1.4992, 1.6710])
tensor([[3.3501, 3.1389, 3.6455],
        [2.9278, 2.4992, 2.6710]])
tensor([3.3501, 3.1389, 3.6455, 2.9278, 2.4992, 2.6710])


如果我们想返回一个真正新的副本（即不共享data内存）该怎么办呢？  
Pytorch还提供了一个**reshape()** 方法可以改变形状，但是此函数并不能摆正返回的是其拷贝，  
所以不推荐使用。推荐先用**clone()** 创造一个副本然后再使用**view()**。

In [20]:
x_cp = x.clone().view(6)
x -= 1
print(x)
print(x_cp)

tensor([[2.3501, 2.1389, 2.6455],
        [1.9278, 1.4992, 1.6710]])
tensor([3.3501, 3.1389, 3.6455, 2.9278, 2.4992, 2.6710])


## 广播机制

当我们对两个形状不同的Tensor按元素运算时，可能会触发**广播（broadcasting）机制** :  
先适当复制元素使这两个Tensor形状相同后再按元素运算。例如

In [21]:
x = torch.arange(1,3).view(1,2)
print(x)
y = torch.arange(1,4).view(3,1)
print(y)
print(x + y)

tensor([[1, 2]])
tensor([[1],
        [2],
        [3]])
tensor([[2, 3],
        [3, 4],
        [4, 5]])


由于x和y分别是1行2列和3行2列的矩阵，如果要计算x + y，那么x中第一行的2个元素被广播（复制）到来
第二行和第三行，而y中第一列的3个元素被广播（复制）到了第二列。如此，就可以对2个3行2列的矩阵按元素相加。

## Tensor和NumPy相互转换

我们可以使用**numpy()** 和**from_numpy()** 将Tensor和NumPy中的数组相互转换。但是需要注意的一点
是：**这两个函数所产生的Tensor和NumPy中的数组共享相同的内存（所以他们之间的转换很快），改变其中一个时另一个也会改变！**   
还有一个常用的将NumPy中的array转换成Tensor的方法就是torch.tensor()，需要注意的是，此方法总是会进行数据拷贝（就会消耗更多的时间和空间），所以返回的Tensor和原来的数据不再共享内存。

Tensor转NumPy数组

In [22]:
a = torch.ones(3)
b = a.numpy()
print(a,b)

a += 1
print(a,b)
b += 1
print(a,b)


tensor([1., 1., 1.]) [1. 1. 1.]
tensor([2., 2., 2.]) [2. 2. 2.]
tensor([3., 3., 3.]) [3. 3. 3.]


NumPy数组转Tensor

In [23]:
import numpy as np
a = np.ones(3)
b = torch.from_numpy(a)
print(a,b)

a += 1
print(a,b)
b += 1
print(a,b)


[1. 1. 1.] tensor([1., 1., 1.], dtype=torch.float64)
[2. 2. 2.] tensor([2., 2., 2.], dtype=torch.float64)
[3. 3. 3.] tensor([3., 3., 3.], dtype=torch.float64)


In [24]:
# 使用torch.tensor()将NumPy数组转换成Tensor(不再共享内存)
c = torch.tensor(a)
a += 1
print(a,c)


[4. 4. 4.] tensor([3., 3., 3.], dtype=torch.float64)


## Tensor on GPU

用方法to()可以将Tensor在CPU和GPU(需要硬件支持)之间相互移动

In [28]:
# 以下代码只有在PyTorch GPU版本上才会执行
if torch.cuda.is_available():
    device = torch.device("cuad")  #GPU
    y = torch.ones_like(x,device=device)
    x = x.to(device)
    z = x + y
    print(z)
    print(z.to("cpu",torch.double))
else:
    print(torch.device("cpu"))

cpu


## 自动求梯度

在深度学习中，我们经常需要对函数求梯度（gradient）。PyTorch提供的**autograd** 包能够根据输入和前向传播过程
自动构建计算图，并执行反向传播。   
  
  
**Tensor** 是**autograd** 包的核心类，如果将其属性 **.requires_grad** 设置为**True** ，它将开始追踪（track）在其上的所有操作（这样就可以利用链式法则进行梯度传播了）。完成计算后，可以调用 **.backward()** 来完成所有梯度计算。此 **Tensor** 的梯度将累积到 **.grad** 属性中。   
  
 如果不想要被继续追踪，可以调用 **.detach()** 将其从追踪记录中分离出来，这样就可以防止将来的计算被追踪，这样梯度就传不过去了。此外，还可以用 **with torch.no_grad()** 将不想被追踪的操作代码块包裹起来，这种方法在评估模型的时候很常用，因为在评估模型时，我们并不需要计算可训练参数（**requires_grad=True** ）的梯度。  
   
 **Function** 是另外一个很重要的类。**Tensor** 和 **Function** 互相结合就可以创建一个记录有整个计算过程的有向无环图（DAG）。每个**Tensor** 都有一个 **.grad_fn** 属性，该属性即创建该**Tensor** 的**Function** ,就是说该**Tensor** 是不是通过某些运算得到的，若是，则**grad_fn** 返回一个与这些运算相关的对象，否则是None。

### <font color="#0000dd">Tensor</font><br /> 

创建一个**Tensor** 并设置**requires_grad=True**:

In [29]:
x = torch.ones(2,2,requires_grad=True)
print(x)
print(x.grad_fn)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
None


做运算操作

In [36]:
y = x + 2
print(y)
print(y.grad_fn)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


更复杂的运算操作:

In [37]:
z = y * y * 3
out = z.mean()
print(z)
print(out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


通过 **.requires_grad_()** 来用in-place的方式改变**requires_grad** 属性：

In [38]:
a = torch.randn(2,2) #缺失情况下默认requires_grad = False
a = ((a * 3)/(a - 1))
print(a.requires_grad) #False
a.requires_grad_(True)
print(a.requires_grad) #True
b = (a * a).sum()
print(b.grad_fn)

False
True


### <font color="#0000dd">梯度</font><br /> 

In [45]:
x = torch.ones(2,2,requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()
out.backward() #等价于out.backward(torch.tensor(1.))

print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


注意：grad在反向传播过程中是**累加** 的（accumulated）,这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前都需要把梯度**清零**

In [48]:
# 再来反向传播一次，注意grad是累加的
out2 = x.sum()
out2.backward()
print(x.grad)

out3 = x.sum()
x.grad.data.zero_()
out3.backward()
print(x.grad)

tensor([[6.5000, 6.5000],
        [6.5000, 6.5000]])
tensor([[1., 1.],
        [1., 1.]])


注意在**y.backward()** 时，如果y是标量,则不需要为backward()传入任何参数；否则，需要传入一个与y同形的Tensor。这样的原因简单来说就是为了避免向量（甚至更高维张量）对张量求导，因此转换成标量对张量进行求导。

* **y.backward()** 的具体例子

In [52]:
x = torch.tensor([1.0,2.0,3.0,4.0],requires_grad=True)
y = 2 * x
z = y.view(2,2)
print(z)


tensor([[2., 4.],
        [6., 8.]], grad_fn=<ViewBackward>)


* **此时z不是一个标量，所以在调用backward()时需要传入一个和z同形的权重向量进行加权求和来得到一个标量。**

In [53]:
v = torch.tensor([[1.0,0.1],[0.01,0.001]],dtype=torch.float)
z.backward(v)
print(x.grad)

tensor([2.0000, 0.2000, 0.0200, 0.0020])
